In [ ]:
#hide
#default_exp export

# nbprocess.export
- Exporting a notebook to a library

In [ ]:
#export
from nbprocess.read import *
from nbprocess.maker import *

from nbprocess.imports import *
from fastcore.script import *
from fastcore.imports import *

from collections import defaultdict
from pprint import pformat
import ast,contextlib

In [ ]:
from fastcore.test import *
from pdb import set_trace
from importlib import reload
import shutil

## NotebookProcessor -

Special comments at the start of a cell can be used to provide information to `nbprocess` about how to process a cell, so we need to be able to find the location of these comments.

In [ ]:
minimal = read_nb('../tests/minimal.ipynb')

In [ ]:
#export
def extract_comments(ss):
    "Take leading comments from lines of code in `ss`, remove `#`, and split"
    ss = ss.splitlines()
    first_code = first(i for i,o in enumerate(ss) if not o.strip() or re.match('\s*[^#\s]', o))
    return L((s.strip()[1:]).strip().split() for s in ss[:first_code]).filter()

nbprocess comments start with `#`, followed by whitespace delimited tokens, which `extract_comments` extracts from the start of a cell, up until a blank line or a line containing something other than comments:

In [ ]:
exp  = "#export module\n# hide\n1+2\n#foo\n#bar"
test_eq(extract_comments(exp), [['export', 'module'],['hide']])

In [ ]:
#export
class NotebookProcessor:
    "Base class for nbprocess notebook processors"
    def __init__(self, path, debug=False): self.nb,self.path,self.debug = read_nb(path),Path(path),debug

Subclass `NotebookProcessor` to add methods to act on nbprocess comments. The method names are of the form `cmd_type`, where "`cmd`" is the first word of the nbprocess comment, and `type` is the `cell_type` of the cell (normally "`code`). The methods must take at least `comment` and `code` as params, plus extra params for any additional words included in a comment. Here's an example that prints any word following a "print me" comment:

In [ ]:
class _PrintExample(NotebookProcessor):
    def printme_code(self, comment, code, to_print): print(to_print)

We can create a processor by passing it a notebook:

In [ ]:
everything_fn = '../tests/01_everything.ipynb'
proc = _PrintExample(everything_fn)

The basic functionality of a notebook processor is to read and act on nbprocess comments.

In [ ]:
#export
@patch
def process_comment(self:NotebookProcessor, comment, cell):
    cmd,*args = comment
    cmd = f"{cmd}_{cell.cell_type}"
    if self.debug: print(cmd, args)
    if not hasattr(self, cmd): return
    try: getattr(self,cmd)(comment,cell, *args)
    except TypeError: pass

Behind the scenes, `process_comment`  is used to call subclass methods. You can subclass this to change the behavior of a processor.

In [ ]:
proc.process_comment(["printme","hello"], SimpleNamespace(cell_type="code"))

hello


In [ ]:
#export
@patch
def process_cell(self:NotebookProcessor, cell):
    comments = extract_comments(cell.source)
    if not comments: return self.no_cmd(cell)
    for comment in comments: self.process_comment(comment, cell)
    return cell

@patch
def no_cmd(self:NotebookProcessor, cell): return cell

Subclass `process_cell` to change how `process_comment` is called. By default, it calls `self.no_cmd` for any cells without comments. The return value of `process_cell` is used to replace the cell in the notebook.

In [ ]:
def _make_code_cell(code, idx=0): return AttrDict(source=code, cell_type="code")
def _make_code_cells(*ss): return dict2nb({'cells':L(ss).map(_make_code_cell)}).cells

proc.process_cell(_make_code_cell("#printme hello"));

hello


In [ ]:
#export
@patch
def process(self:NotebookProcessor):
    "Process all cells with `process_cell` and replace `self.nb.cells` with result"
    for i in range_of(self.nb.cells): self.nb.cells[i] = self.process_cell(self.nb.cells[i])

In [ ]:
proc.process()

testing


`NotebookProcessor.process` doesn't change a notebook or act on any comments, unless you subclass it.

In [ ]:
everything = read_nb(everything_fn)
proc = NotebookProcessor(everything_fn)
proc.process()
for a_,b_ in zip(everything.cells, proc.nb.cells): test_eq(str(a_),str(b_))

## ExportModuleProcessor -

In [ ]:
#export
class ExportModuleProcessor(NotebookProcessor):
    "A `NotebookProcessor` which exports code to a module"
    def __init__(self, path, dest, mod_maker=ModuleMaker, debug=False):
        dest = Path(dest)
        store_attr()
        super().__init__(path,debug=debug)

    def process(self):
        self.modules,self.in_all = defaultdict(L),defaultdict(L)
        super().process()

Specify `path` containing the source notebook, `dest` where the module(s) will be exported to, and optionally a class to use to create the module (`ModuleMaker`, by default).

In [ ]:
proc = ExportModuleProcessor(everything_fn, 'tmp')

In [ ]:
#export
@patch
def default_exp_code(self:ExportModuleProcessor, comment, cell, exp_to): self.default_exp = exp_to

You must include a `default_exp` comment somewhere in your notebook to show what module to export to by default.

In [ ]:
proc.process()
test_eq(proc.default_exp, 'everything')

In [ ]:
#export
@patch
def exporti_code(self:ExportModuleProcessor, comment, cell, exp_to=None):
    "Export a cell, without including the definition in `__all__`"
    mod = ifnone(exp_to, '#')
    self.modules[mod].append(cell)
    return mod

Exported cells are stored in a `dict` called `modules`, where the keys are the modules exported to. Those without an explicit module are stored in the `'#'` key, which will be exported to `default_exp`.

In [ ]:
proc.process()
proc.modules['#']

(#1) [#exporti
#just another comment
def c_y_nall(): ...]

In [ ]:
#export
@patch
def export_code(self:ExportModuleProcessor, comment, cell, exp_to=None):
    "Export a cell, adding the definition in `__all__`"
    mod = self.exporti_code(comment, cell, exp_to=exp_to)
    self.in_all[mod].append(cell)

In [ ]:
@patch
def exports_code(self:ExportModuleProcessor, comment, cell, exp_to=None):
    "Same as `export_code`, but also show source code in docs"
    self.export_code(comment, cell, exp_to=exp_to)

In [ ]:
#export
@patch
def create_modules(self:ExportModuleProcessor):
    "Create module(s) from notebook"
    self.process()
    for mod,cells in self.modules.items():
        all_cells = self.in_all[mod]
        name = self.default_exp if mod=='#' else mod
        mm = self.mod_maker(dest=self.dest, name=name, nb_path=self.path, is_new=mod=='#')
        mm.make(cells, all_cells)

Let's check we can import a test file:

In [ ]:
shutil.rmtree('tmp')
proc = ExportModuleProcessor('../tests/00_some.thing.ipynb', 'tmp')
proc.create_modules()

import tmp.some.thing
reload(tmp.some.thing)
test_eq(tmp.some.thing.__all__, ['a'])
test_eq(tmp.some.thing.a, 1)

We'll also check that our 'everything' file exports correctly:

In [ ]:
proc = ExportModuleProcessor(everything_fn, 'tmp')
proc.create_modules()

import tmp.everything
reload(tmp.everything)
from tmp.everything import *
g = globals()
_alls = L("a b d e m n o p q".split())
for s in _alls.map("{}_y"): assert s in g, s
for s in "c_y_nall _f_y_nall g_n h_n i_n j_n k_n l_n".split(): assert s not in g, s
for s in _alls.map("{}_y") + ["c_y_nall", "_f_y_nall"]: assert hasattr(tmp.everything,s), s

That notebook should also export one extra function to `tmp.some.thing`:

In [ ]:
import tmp.some.thing
reload(tmp.some.thing)
test_eq(tmp.some.thing.__all__, ['a','h_n'])
test_eq(tmp.some.thing.h_n(), None)

## Export -

In [ ]:
path = Path('../nbprocess')
(path/'export.py').unlink(missing_ok=True)
ExportModuleProcessor('02_export.ipynb', 'nbprocess').create_modules()
import nbprocess.export
reload(nbprocess.export)
assert hasattr(nbprocess.export, 'ModuleMaker')

In [ ]:
ExportModuleProcessor('00_read.ipynb', 'nbprocess').create_modules()
ExportModuleProcessor('01_maker.ipynb', 'nbprocess').create_modules()

In [ ]:
# #hide
# def nb2dict(d, k=None):
#     "Convert parsed notebook to `dict`"
#     if k=='source': return d.splitlines(keepends=True)
#     if isinstance(d, (L,list)): return list(L(d).map(nb2dict))
#     if not isinstance(d, dict): return d
#     return dict(**{k:nb2dict(v,k) for k,v in d.items() if k[-1] != '_'})

# # This returns the exact same string as saved by Jupyter.

# assert minimal_txt==nb2dict(minimal)

# #export
# def write_nb(nb, path):
#     "Write `nb` to `path`"
#     nb = nb2dict(nb)
#     with io.open(path, 'w', encoding='utf-8') as f:
#         f.write(json.dumps(nb, sort_keys=True, indent=1, ensure_ascii=False))
#         f.write("\n")